In [ ]:
from operator import index
import torch
from torch_sparse import SparseTensor
from torch import tensor

from torch_sparse import SparseTensor, matmul


row = torch.tensor([0, 1, 2, 2, 2, 3, 3, 4])
col = torch.tensor([2, 2, 0, 1, 3, 2, 4, 3])
c = SparseTensor(row=row, col=col)

print(c)

adjk = matmul(c, c)
print(adjk)

In [ ]:
print(aa[0].unsqueeze(-1).dim())

print(aa.device)

In [6]:
from torch_sparse import SparseTensor, matmul

# plot results 
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from matplotlib import cm
from matplotlib.colors import ListedColormap
import networkx as nx

# here..
cmaps = {}

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

x_y_label_font = 20
x_y_legend_font = 20

plt.rc('font', family='Times New Roman')
fig_dpi = 220
fig_shape_squre = (6, 5)

def plot_color_gradients(category, cmap_list):
    # Create figure and adjust figure height to number of colormaps
    nrows = len(cmap_list)
    figh = 0.35 + 0.15 + (nrows + (nrows - 1) * 0.1) * 0.22
    fig, axs = plt.subplots(nrows=nrows + 1, figsize=(6.4, figh), dpi=100)
    fig.subplots_adjust(top=1 - 0.35 / figh, bottom=0.15 / figh,
                        left=0.2, right=0.99)
    axs[0].set_title(f'{category} colormaps', fontsize=14)

    for ax, name in zip(axs, cmap_list):
        ax.imshow(gradient, aspect='auto', cmap=plt.get_cmap(name))
        ax.text(-0.01, 0.5, name, va='center', ha='right', fontsize=10,
                transform=ax.transAxes)

    # Turn off *all* ticks & spines, not just the ones with colormaps.
    for ax in axs:
        ax.set_axis_off()

    # Save colormap list for later.
    cmaps[category] = cmap_list
    plt.show()


class MyColor(object):
    def __init__(self, cmap_name, skip_idx=5, backup_name='Set1', 
                 backup_color=3, add_red=False, pre_defined=False):
        if pre_defined:
            # colors = ['#3682be','#45a776','#f05326','#eed777','#334f65','#b3974e','#38cb7d','#ddae33','#844bb3','#93c555','#5f6694','#df3881']
            colors = '00a8e1 - 99cc00 - e30039 - 800080 - 00994e - ff6600 - 808000 - db00c2 - 008080 - 0000ff - c8cc00'
            colors = ["#"+c.strip() for c in colors.split('-')]
            print(colors)
            cmap = ListedColormap(colors, name = 'mycmap')
            self.color_set = list(cmap.colors)
        else:
            if isinstance(cmap_name, list):
                #NOTE: combine all cmaps:
                self.color_set = []
                for cname in cmap_name:
                    self.color_set.extend(list(plt.get_cmap(cname).colors))
                print('color_set: ', self.color_set[15])
            else:
                self.color_set = list(plt.get_cmap(cmap_name).colors)
                
        # NOTE: always ignore light yellow in Set1

        if add_red:
            self.light_set = list(plt.get_cmap('Set1').colors)
            self.color_set  = [v for i, v in enumerate(list(plt.get_cmap(cmap_name).colors)[:-1]) if i!=5]
            self.color_set.extend([self.light_set[0]])
            self.color_set.extend([self.light_set[4]])

        self.backup_set = plt.get_cmap(backup_name).colors
        self.backup_color = backup_color
        self.skip_idx=skip_idx
        self.idx = 0
        self.color_len = len(self.color_set)
        
    def get_color(self):
        if self.idx == self.color_len - 1:
            self.idx = 0
        if self.idx == self.skip_idx:
            self.idx += 1
            return self.backup_set[self.backup_color]
        color = self.color_set[self.idx]
        self.idx += 1
        return color
    
    def shuffle(self):
        np.random.shuffle(self.color_set)
    

plot_color_gradients('Qualitative',
                     ['Pastel1', 'Pastel2', 'Paired', 'Accent', 'Dark2',
                      'Set1', 'Set2', 'Set3', 'tab10', 'tab20', 'tab20b',
                      'tab20c'])


<Figure size 640x338.2 with 13 Axes>

In [18]:
import pickle as pk



class StandardScaler():

    def __init__(self, mean, std, fill_zeroes=False):
        self.mean = mean
        self.std = std
        self.fill_zeroes = fill_zeroes

    def transform(self, data):
        if self.fill_zeroes:
            mask = (data == 0)
            data[mask] = self.mean
            
        if isinstance(data, list):
            if self.std == 0:
                return [0.01 for d in data]
            
            return [(d - self.mean)/self.std for d in data]
        
        if self.std == 0:
            return np.zeros_like(data) + 0.01
        
        return (data - self.mean) / self.std

    def inverse_transform(self, data):
        return (data * self.std) + self.mean


def normalize(data, along_axis=None, ignore_norm=[], same_data_shape=True):
    '''
        only norm numpy type data with last dimension.
    '''
    if isinstance(data, list):
        if same_data_shape:
            # hear also along each axis:
            cur_data = np.array(data).reshape(-1, 1)
            return normalize(cur_data, along_axis=along_axis, ignore_norm=ignore_norm)
        else:
        # NOTE: data shape: [(N, C), (N1, C),...], so cannot concatenate
            normed_res = []
            for each in data:
                each_norm = normalize(each, along_axis=along_axis, ignore_norm=ignore_norm)
                normed_res.append(each_norm)
            return normed_res
        
    if not isinstance(data, np.ndarray):
        data = data.cpu().numpy()
        
    if along_axis is not None:
        if along_axis == -1:
            # along all axis separately. data shape:(NxC) along each C_i
            for ax in range(data.shape[-1]):
                if ax in ignore_norm or (ax - data.shape[-1]) in ignore_norm:
                    continue
                print('data shape: ', data.shape)
                mean = np.mean(data[:, ax])
                std = np.std(data[:, ax])
                scaler = StandardScaler(mean=mean, std=std)
                data[:, ax] = scaler.transform(data[:, ax])
            return data
        else:
            print('norm along ', along_axis)
            pass
            
    mean = np.mean(data)
    std = np.std(data)
    scaler = StandardScaler(mean=mean, std=std)
    normed_data = scaler.transform(data)
                
    return normed_data, scaler


# Load all datasets:

import sys,os
sys.path.append(os.getcwd())


import my_utils

from PrepareDatasets import DATASETS

with open(f'DATA/syn_degree_0.9_class10.pkl')
data = pk.load()
    

graph_path ='./DATA/syn_degree/processed/graphwise_syn_degree_add_avg_degree.pkl'
# load as pickle:
with open(graph_path, 'rb') as f:
    a = pk.load(f)
    plt.figure()
    graph_features = normalize(a, along_axis=-1)
    aa = [i.item() for i in graph_features]
    
    aa = sorted(aa)
    plt.plot(aa)
    # not normalized.
    
    


processed_dir:  DATA/syn_degree/processed
load dataset !
SynDataset load data_path: DATA/syn_degree_0.9_class10.pkl


FileNotFoundError: [Errno 2] No such file or directory: 'DATA/syn_degree_0.9_class10.pkl'